# Frozen Lake Challenge

This notebook has been broken out into two parts:

* An explaination of Q-Learning - the algorithm we'll be using
* An various implementations of Q-Learning to solve the challenge

## Understanding Reinforcement Learning In General

The central idea of reinforcement learning, is to learn a strategy over time, based on interacting with an environment. There are strong correlaries between this notion of environment interaction and graph traversal. For this assignment I'll make use of Q learning to understand the basics of reinforcement learning. There are other techniques I could have employed, namely SARSA and Temporal difference learning, Policy Gradients, Deep Q-learning, Advantage Advantage Actor Critic, Proximal Policy Gradients, or Random Network Distillation.

## Understanding Q-Learning In Specific

The basic idea of Q learning is we start with a set of possible choices, and if we've seen all those choices before, we choose the choice that is the best (based on past experience).  The way the program evaluates which new possible choice is the best, is via looking at the rewards associated with each choice.  Let's look at a method to evaluate a set of choices, in the abstract:

```
def choose_action(state,actions,representation):
    q = [evaluate(representation,(state,action)) for action in actions]
    q = remove_all(q,None)
    if q == []: return random.choice(actions)
    else:
        max_q = max(q)
        num_equally_valued_actions = q.count(max_q)
        if num_equally_valued_actions > 1:
            possible_choices = [i for i in range(len(actions)) if q[i] == max_q]
            action_choice = random.choice(possible_choices)
        else:
            action_choice = q.index(max_q)
        return actions[action_choice]
```

Here state, represents the current state of the "player", the actions represent the possible actions, and the representation is the model rewards associated with each state,action pair.  Using the representation, the program chooses which action to execute next.  Here the representation is a dictionary, with keys as state,action pairs, and values as the associated rewards.  

The code should be fairly straight forward: 

The full set of possible actions is observed and their associated rewards are recoded in a list: 

`q = [evaluate(representation,(state,action)) for action in actions]`

This gives us a list "q" which gives us all the relevant rewards, indexed by possible actions.  We then choose the action which yields the greatest reward:

`max_q = max(q)`

If there are multiple rewards that yield the best possible scenario, we simply randomly choose one such reward.  If we have no information about our action set, we simply choose an action at random: 

`if q == []: return random.choice(actions)`

This is more or else the crux of basic Q-Learning.  

Now let's dig in a little bit to how our representation get's updated:

```
def update_representation(representation,state,action,reward,alpha=0.3):
    try:
        current_value = representation[(state,action)]
    except:
        current_value = None
    if current_value:
        representation[(state,action)] = current_value + alpha * reward
    else:
        representation[(state,action)] = reward
    return representation
```

Here we do the most naive thing possible, simply discounting the present state against any past state.  In this way, our first impression dominates and any future iterations simply contribute to a lesser extent.  Over many repeated plays, this tends to cancel out and the long term trend will dominate eventually.

### Playing with update_representation

The long term representation that we use, after many iterations is very important to the learning mechanism we employ.  Intuitively, we can think of this long term representation as a `policy`.  By finding the right way to generate long term views, we can have significant gains in how our AI performs.  

Let's look at some alternative ways to update our representation:

```
def update_representation(representation,counts,state,action,reward,alpha=0.3):
    try:
        current_value = representation[(state,action)]
    except:
        current_value = None
    if current_value:
    qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        representation[(state,action)] = current_value + (1/counts[(state,action)]*( reward - current_value ))
        counts[(state,action)] += 1
    else:
        counts[(state,action)] = 1
        representation[(state,action)] = reward
    return representation,counts
```

In the above code instead of simply allowing our first value to dominate (until the long term), we use the average value of the representation.  Here the current value is the average up until this point and the somewhat confusing looking thing:

`(1/counts[(state,action)]*( reward - current_value ))`

Is the updated additive contribution of the reward: `reward - current_value`, discounted by the total number of terms seen `1/counts[(state,action)]`.  The reason that we need the count for each state,action combination is because we want the average for each state,action pair, rather than the overall state.  However, I assure you, this is the average.  

A more natural, albeit less space efficient version of this code is below:

```
def update_representation(representation,counts,state,action,reward,alpha=0.3):
    try:
        current_value = representation[(state,action)]
    except:
        current_value = None
    if current_value:
	    counts[(state,action)] += [reward]
        representation[(state,action)] = sum(counts[(state,action)])/float(len(counts[(state,action)]))
    else:
        counts[(state,action)] = [reward]
        representation[(state,action)] = reward
    return representation,counts
```

## Let's look at the full code

Now that we have everything all set up - let's look at how the train and play algorithms work in general.  We train the AI to play the game:

```
def train(board,iterations):
    representation = {}
    counts = {}
    state = (0,0)
    score = 0
    actions = ("up","down","left","right")
    for _ in range(iterations):
        while score < 100:
            states = generate_new_board_states(board)
            action = choose_action(state,actions,representation)
            state = update_state(action,state,len(states),len(states[0]))
            representation,counts = update_representation(representation,counts,state,action,states[state[0]][state[1]],alpha=0.3)
            score = sum([elem for elem in representation.values() if elem > 0])
    return representation
```

## Solving the Frozen Lake Challenge

So far we've looked at a few ways of how to do Q-Learning, specifically a couple of ways we could update our reward over the long term.  Now let's go ahead and solve the problem in the frozen lake challenge - teaching an agent how to get over the frozen lake safely.  

For this we'll make use of numpy, gym and random - as shown below:

In [2]:
import numpy as np
import gym
import random

## Step 1 - Setting up the environment

Since we are using the open gym environment, we'll set that up first:

In [3]:
env = gym.make("FrozenLake-v0")

## Step 2 - Setting up our Q-Table

The formal name of the representation object we showed above is called a q-table.  The open gym framework tells us ahead of time how big our q-table should be.  So we can set it up ahead of time as follows:

In [4]:
action_size = env.action_space.n
state_size = env.observation_space.n
qtable = np.zeros((state_size, action_size))

## Step 3 - Implementing Q-Learning In Numpy (instead of vanilla Python)

Here we will make use of the policy based algorithm I've implemented above to train the agent.  You'll note that in this version we've added more tuning parameters to the policy engine - this is to improve performance in the model and to avoid overfitting.

In [19]:
def choose_action(qtable, state, epsilon):
    if random.uniform(0, 1) > epsilon:
        # exploitation based on previous knowledge
        action = np.argmax(qtable[state,:])
    else:
        # exploration to avoid local maxima
        action = env.action_space.sample()
    return action

def update_qtable(qtable, action, reward, state, new_state, learning_rate, alpha):
    best_choice = alpha * np.max(qtable[new_state, :])
    discount_rate = qtable[state, action]
    reward_update = reward + best_choice - discount_rate
    qtable[state, action] = qtable[state, action] + learning_rate * reward_update
    return qtable

def update_epsilon(min_epsilon, max_epsilon, decay_rate, episode):
    return min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    
def train_agent(qtable, total_episodes, learning_rate, max_steps, 
                alpha=0.95, epsilon=1.0, max_epsilon=1.0, min_epsilon=0.01, decay_rate=0.005):

    rewards = []
    for episode in range(total_episodes):
        state = env.reset()
        done = False
        total_rewards = 0

        for step in range(max_steps):
            action = choose_action(qtable, state, epsilon)
            new_state, reward, done, info = env.step(action)
            qtable = update_qtable(qtable, action, reward,
                                   state, new_state, learning_rate, alpha)
            total_rewards += reward
            state = new_state
            if done == True: 
                break

        # Reduce epsilon because we need less and less exploration
        epsilon = update_epsilon(min_epsilon, max_epsilon, decay_rate, episode)
        rewards.append(total_rewards)
    return qtable, rewards

total_episodes = 15000        
learning_rate = 0.8           
max_steps = 99                

action_size = env.action_space.n
state_size = env.observation_space.n
qtable = np.zeros((state_size, action_size))
import time
start = time.time()
qtable, rewards = train_agent(qtable, total_episodes, learning_rate, max_steps, alpha=0.95, epsilon=1.0, max_epsilon=1.0, min_epsilon=0.01, decay_rate=0.005)
print(time.time() - start)
print ("Score", sum(rewards)/total_episodes)
print(qtable)


8.157054901123047
Score 0.47833333333333333
[[2.20187776e-01 7.34658517e-02 7.47754899e-02 4.79122658e-02]
 [8.73288881e-03 1.49991007e-02 9.48207847e-04 2.68411616e-01]
 [6.09828802e-03 7.66410342e-03 8.25722079e-03 2.40696396e-01]
 [2.03516613e-04 5.98210846e-03 1.73667862e-03 2.23190989e-02]
 [4.27998797e-01 7.54666868e-02 3.23511377e-02 4.28425720e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.07960956e-06 3.81691048e-09 1.08954583e-02 1.30138740e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [6.97529044e-02 6.91885817e-02 7.85186578e-02 5.90557999e-01]
 [1.11564972e-02 7.90597195e-01 1.19240052e-01 7.39234946e-02]
 [2.15434725e-01 8.42559502e-04 1.02602869e-04 3.43730764e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.99148855e-02 1.92397347e-02 8.59852709e-01 9.31978732e-02]
 [3.60315504e-01 9.89482037e-01 2.34672977e-01 1.53284346e-01]
 [0.0000000

## Tuning Our Agent's Parameters

Now that we have a working agent, let's try to maximize our score by tuning some of the parameters we are passing into the `train_agent` function.

In [18]:
import itertools

total_episodes = 16000        
learning_rate_range = [0.8, 0.9]           
max_steps_range = 110                
alpha_range = [0.8, 0.9, 0.99] 
epsilon = 1.0
max_epsilon=1.0 
min_epsilon_range=[0.01, 0.05, 0.1] 
decay_rate_range=[0.001, 0.005, 0.01]

qtables = []
scores = []
for parameters in itertools.product(learning_rate_range, alpha_range, min_epsilon_range, decay_rate_range):
    
    learning_rate = parameters[0]
    alpha = parameters[1]
    min_epsilon = parameters[2]
    decay_rate = parameters[3]
    
    action_size = env.action_space.n
    state_size = env.observation_space.n
    qtable = np.zeros((state_size, action_size))

    qtable, rewards = train_agent(qtable, 
                                  total_episodes, 
                                  learning_rate, 
                                  max_steps, 
                                  alpha=0.95, 
                                  epsilon=1.0, 
                                  max_epsilon=1.0, 
                                  min_epsilon=0.01, 
                                  decay_rate=0.005)
    scores.append(sum(rewards)/total_episodes)
    qtables.append(qtable)
    
best_score = 0
best_qtable = None
for index, score in enumerate(scores):
    if score > best_score:
        best_score = score
        best_qtable = qtables[index]
print(best_score)

0.4921875


# Now let's play the game!

Now that we are done training our agent to be the best possible, we can play our game with the resultant agent, stored in the qtable:

In [21]:
env.reset()
for episode in range(10):
    state = env.reset()
    step = 0
    done = False
    print("EPISODE ", episode)
    print()

    for step in range(max_steps):
        
        action = np.argmax(best_qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            env.render()
            
            print("Number of steps", step)
            break
        state = new_state
env.close()

EPISODE  0

EPISODE  1

  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 48
EPISODE  2

  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 21
EPISODE  3

EPISODE  4

EPISODE  5

  (Right)
SFFF
FHFH
FFFH
HFFG
Number of steps 16
EPISODE  6

  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 23
EPISODE  7

  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 21
EPISODE  8

  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 27
EPISODE  9

  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 54
